In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import datatable as dt
import multiprocessing
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
train = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()

In [ ]:
train['action'] = ((train['resp'] > 0) * 1)

Utility score function

In [ ]:
def utility_scoring(df):
    """
    To get the utility score used in the challenge.
    """
    from math import sqrt
    u = 0
    Pi = []
    Pis = []
    count_i = len(df['date'].unique())
    for i in list(df['date'].unique()):
        #print("date value= ", i)
        #print(10*"=")
        tmp = df[df['date'] == i][["date","weight","resp","action"]]
        tmp["mult"] = tmp['weight'] * tmp['resp'] * tmp['action']
        Pi.append(tmp["mult"].sum())
        Pis.append((tmp["mult"].sum())**2)
    
    t =  sum(Pi)/sqrt(sum(Pis)) * sqrt(250/count_i)
    u = min(max(t,0),6)*sum(Pi)
    return u

creating a dataframe for a day and its score

In [ ]:
_score = {'date':[],'score':[]}

In [ ]:
%%time
for i in train['date'].unique():
    df = train.query(f"date == {i}")
    _score['date'].append(i)
    _score['score'].append(utility_scoring(df))

In [ ]:
score_df = pd.DataFrame.from_dict(_score)

In [ ]:
px.bar(score_df, x='date', y='score')

Now lets see how many traders were done for each day in train data

In [ ]:
ts_id = train.groupby('date')['ts_id'].count().reset_index()

In [ ]:
merged_df = score_df.merge(ts_id, on='date', how='left')

Sort data based on score from merged dataframe and select top 10

In [ ]:
merged_df.sort_values('score', ascending=False).head(10)

Let's break the graph for each 100 days

In [ ]:
px.bar(merged_df.query("date > 0 and date < 100"), x='date', y='score')

In [ ]:
px.bar(merged_df.query("date > 100 and date < 200"), x='date', y='score')

In [ ]:
px.bar(merged_df.query("date > 400 and date < 500"), x='date', y='score')

lets calculate mean for these periods

In [ ]:
merged_df.query("date > 0 and date < 100")['score'].mean()

In [ ]:
merged_df.query("date > 100 and date < 200")['score'].mean()

In [ ]:
merged_df.query("date > 200 and date < 300")['score'].mean()

In [ ]:
merged_df.query("date > 300 and date < 400")['score'].mean()

In [ ]:
merged_df.query("date > 400 and date < 500")['score'].mean()

We can calculate std and variance on these blocks as well

In [ ]:
merged_df.query("date > 0 and date < 100")['score'].var(), merged_df.query("date > 0 and date < 100")['score'].std()

In [ ]:
merged_df.query("date > 100 and date < 200")['score'].var(), merged_df.query("date > 100 and date < 200")['score'].std()

In [ ]:
merged_df.query("date > 200 and date < 300")['score'].var(), merged_df.query("date > 200 and date < 300")['score'].std()

In [ ]:
merged_df.query("date > 300 and date < 400")['score'].var(), merged_df.query("date > 300 and date < 400")['score'].std()

In [ ]:
merged_df.query("date > 400 and date < 500")['score'].var(), merged_df.query("date > 400 and date < 500")['score'].std()

Now let's remove the data where weight is zero and follow same process and see var and std

In [ ]:
train.shape

In [ ]:
train = train.query("weight !=0")

In [ ]:
train.shape

In [ ]:
_score_1 = {'date':[],'score':[]}

In [ ]:
for i in train['date'].unique():
    df = train.query(f"date == {i}")
    _score_1['date'].append(i)
    _score_1['score'].append(utility_scoring(df))

In [ ]:
score_df_1 = pd.DataFrame.from_dict(_score_1)

In [ ]:
ts_id_1 = train.groupby('date')['ts_id'].count().reset_index()

In [ ]:
merged_df_1 = score_df_1.merge(ts_id_1, on='date', how='left')

In [ ]:
px.bar(merged_df_1.query("date > 0 and date < 100"), x='date', y='score')

Here score doesn't change for first 100 days but let's see what are the values of var and std

In [ ]:
merged_df_1.query("date > 0 and date < 100")['score'].var(), merged_df.query("date > 0 and date < 100")['score'].std()

In [ ]:
merged_df_1.query("date > 100 and date < 200")['score'].var(), merged_df.query("date > 100 and date < 200")['score'].std()

In [ ]:
merged_df_1.query("date > 400 and date < 500")['score'].var(), merged_df_1.query("date > 400 and date < 500")['score'].std()